# 0. 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 입력 데이터셋 구축

In [ ]:
# === 0) 기본 세팅 ============================================================
import os, json, gc
import pandas as pd
import numpy as np

# === 1) 경로/옵션 =============================================================
HOSP_DIR   = "/content/drive/MyDrive/DILAB/MARS/mimiciv_3.1/files/hosp"
ICU_DIR    = "/content/drive/MyDrive/DILAB/MARS/mimiciv_3.1/files/icu"
NOTE_DIR   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note"

OUTPUT_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv_reconstructed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 테스트/성능 옵션
HADM_LIMIT                 = None     # 검증 후 None
USE_EMAR                   = False
USE_LABS                   = True
USE_MICROBIO               = True
USE_ICU_LIGHT              = True
# ⚠️ A-구조 입력에서는 텍스트 스니펫 불필요
USE_RADIOLOGY_TEXT         = False
USE_ICU_VENT_PRESSOR_HEAVY = False

SAVE_PARQUET = True
SAVE_CSV     = False

# 유틸
def td(dt):  # 안전한 to_datetime
    return pd.to_datetime(dt, errors="coerce", utc=True)

def jdump(x):
    return json.dumps(x, ensure_ascii=False, separators=(",", ":"))

def safe_read_csv(path, usecols=None, dtype=None):
    return pd.read_csv(path, usecols=usecols, dtype=dtype, low_memory=False)

# === 2) 코어 테이블: 환자/입원 =================================================
print("📥 Loading patients/admissions ...")
patients = safe_read_csv(
    f"{HOSP_DIR}/patients.csv",
    usecols=["subject_id","gender","anchor_age","anchor_year","anchor_year_group"]
)

admissions = safe_read_csv(
    f"{HOSP_DIR}/admissions.csv",
    usecols=[
        "subject_id","hadm_id","admittime","dischtime","deathtime","admission_type",
        "admit_provider_id","admission_location","discharge_location","insurance",
        "edregtime","edouttime","hospital_expire_flag"
    ]
)

for c in ["admittime","dischtime","deathtime","edregtime","edouttime"]:
    admissions[c] = td(admissions[c])

core = admissions.merge(patients, on="subject_id", how="left")

if HADM_LIMIT is not None:
    keep_hadm = core["hadm_id"].dropna().unique()[:int(HADM_LIMIT)]
    core = core[core["hadm_id"].isin(keep_hadm)].copy()

print(f"✅ core rows: {len(core):,}")

# === 3) (A 방식) 퇴원요약 텍스트/섹션 완전 스킵 ================================
# (아무 작업도 하지 않음: 노트 텍스트/섹션은 생성 목표에 누설이므로 제외)

# === 4) 처방/EMAR 요약(구조화) ================================================
print("📥 Loading prescriptions ...")
presc_cols = [
    "subject_id","hadm_id","pharmacy_id","poe_id","poe_seq","order_provider_id",
    "starttime","stoptime","drug_type","drug","formulary_drug_cd","gsn","ndc",
    "prod_strength","form_rx","dose_val_rx","dose_unit_rx",
    "form_val_disp","form_unit_disp","doses_per_24_hrs","route"
]
presc = safe_read_csv(f"{HOSP_DIR}/prescriptions.csv", usecols=presc_cols)
presc["starttime"] = td(presc["starttime"])
presc["stoptime"]  = td(presc["stoptime"])

# 현재 core hadm만 + 입원창 겹침
adm_times = core.set_index("hadm_id")[["admittime","dischtime"]].to_dict("index")
presc = presc[presc["hadm_id"].isin(core["hadm_id"])].copy()

def overlaps_adm(row):
    a = adm_times.get(row["hadm_id"])
    if a is None: return False
    s, e = row["starttime"], row["stoptime"]
    if pd.isna(s) or pd.isna(e) or pd.isna(a["admittime"]) or pd.isna(a["dischtime"]):
        return True
    return not (e < a["admittime"] or s > a["dischtime"])

presc = presc[presc.apply(overlaps_adm, axis=1)]

def summarize_prescriptions(df):
    def one_hadm(g):
        items = []
        for _, r in g.iterrows():
            items.append({
                "drug": r["drug"],
                "route": r["route"],
                "dose": (str(r["dose_val_rx"]) if pd.notna(r["dose_val_rx"]) else None),
                "dose_unit": r["dose_unit_rx"],
                "doses_per_24_hrs": r["doses_per_24_hrs"],
                "start": (r["starttime"].isoformat() if pd.notna(r["starttime"]) else None),
                "stop":  (r["stoptime"].isoformat() if pd.notna(r["stoptime"]) else None),
                "drug_type": r["drug_type"],
            })
        return jdump(items)
    return (df.groupby("hadm_id", group_keys=False)
              .apply(lambda g: pd.Series({"inpatient_med_summary_json": one_hadm(g)}))
              .reset_index())

presc_summary = summarize_prescriptions(presc)

# eMAR(선택)
if USE_EMAR:
    print("📥 Loading eMAR ...")
    emar = safe_read_csv(
        f"{HOSP_DIR}/emar.csv",
        usecols=["subject_id","hadm_id","emar_id","emar_seq","poe_id","pharmacy_id","enter_provider_id","charttime","medication","event_txt","scheduletime","storetime"]
    )
    emar["charttime"] = td(emar["charttime"])
    emar = emar[emar["hadm_id"].isin(core["hadm_id"])].copy()

    def summarize_emar(df):
        def one_hadm(g):
            out = {}
            for med, gg in g.groupby("medication"):
                d = {
                    "n_events": int(len(gg)),
                    "n_given": int((gg["event_txt"].str.lower()=="given").sum()),
                    "last_charttime": (gg["charttime"].max().isoformat() if pd.notna(gg["charttime"].max()) else None),
                }
                out[med] = d
            return jdump(out)
        return (df.groupby("hadm_id", group_keys=False)
                  .apply(lambda g: pd.Series({"emar_admin_summary_json": one_hadm(g)}))
                  .reset_index())
    emar_summary = summarize_emar(emar)
else:
    emar_summary = pd.DataFrame(columns=["hadm_id","emar_admin_summary_json"])

# === 5) Labs/Micro (구조화만) ==================================================
if USE_LABS:
    print("📥 Loading d_labitems & labevents (filtered) ...")
    dlab = safe_read_csv(f"{HOSP_DIR}/d_labitems.csv", usecols=["itemid","label","fluid","category"])
    TARGET_LABELS = {
        "WBC":"WBC","Hemoglobin":"Hgb","Platelet Count":"Plt",
        "Sodium":"Na","Potassium":"K","Chloride":"Cl","Bicarbonate":"HCO3",
        "Creatinine":"Cr","Urea Nitrogen":"BUN","Glucose":"Glucose",
    }
    target_dlab = dlab[dlab["label"].isin(TARGET_LABELS.keys())].copy()
    target_dlab["short"] = target_dlab["label"].map(TARGET_LABELS)
    target_itemids = set(target_dlab["itemid"].tolist())
    target_map     = dict(zip(target_dlab["itemid"], target_dlab["short"]))

    lab_summary_rows = []
    cols = ["labevent_id","subject_id","hadm_id","specimen_id","itemid","charttime","value","valuenum","valueuom","ref_range_lower","ref_range_upper","flag","priority"]
    chunk_iter = pd.read_csv(f"{HOSP_DIR}/labevents.csv", usecols=cols, chunksize=1_000_000, low_memory=False)
    keep_hadm_set = set(core["hadm_id"].dropna().unique())
    for chunk in chunk_iter:
        chunk = chunk[chunk["hadm_id"].isin(keep_hadm_set)]
        chunk = chunk[chunk["itemid"].isin(target_itemids)]
        if chunk.empty:
            continue
        chunk["charttime"] = td(chunk["charttime"])
        def in_window(r):
            a = adm_times.get(r["hadm_id"])
            if a is None: return False
            ct = r["charttime"]
            if pd.isna(ct) or pd.isna(a["admittime"]) or pd.isna(a["dischtime"]):
                return True
            return (a["admittime"] <= ct) and (ct <= a["dischtime"])
        chunk = chunk[chunk.apply(in_window, axis=1)]
        if chunk.empty:
            continue
        chunk["short"] = chunk["itemid"].map(target_map)
        agg = (chunk.sort_values("charttime")
                    .groupby(["hadm_id","short"])
                    .agg(min_val=("valuenum","min"),
                         max_val=("valuenum","max"),
                         last_val=("valuenum","last"),
                         unit=("valueuom","last"))
                    .reset_index())
        lab_summary_rows.append(agg); del chunk, agg; gc.collect()

    if lab_summary_rows:
        labs_agg = pd.concat(lab_summary_rows, ignore_index=True)
        def one_hadm(g):
            out = {}
            for _, r in g.iterrows():
                out[r["short"]] = {
                    "min": (None if pd.isna(r["min_val"]) else float(r["min_val"])),
                    "max": (None if pd.isna(r["max_val"]) else float(r["max_val"])),
                    "last":(None if pd.isna(r["last_val"]) else float(r["last_val"])),
                    "unit": r["unit"]
                }
            return jdump(out)
        labs_summary_full = (labs_agg.groupby("hadm_id", group_keys=False)
                               .apply(lambda g: pd.Series({"lab_summary_json": one_hadm(g)}))
                               .reset_index())
    else:
        labs_summary_full = pd.DataFrame(columns=["hadm_id","lab_summary_json"])
else:
    labs_summary_full = pd.DataFrame(columns=["hadm_id","lab_summary_json"])

# (선택) A-입력 경량화: labs를 last+unit만 남기는 함수
def labs_last_only(js):
    if pd.isna(js) or js is None:
        return None
    try:
        obj = json.loads(js) if isinstance(js, str) else js
        out = {}
        for k, d in obj.items():
            if not isinstance(d, dict):
                continue
            last = d.get("last")
            unit = d.get("unit")
            if last is not None:
                out[k] = {"last": last, "unit": unit}
        return jdump(out)
    except Exception:
        return None

if USE_LABS:
    labs_summary = labs_summary_full.copy()
    labs_summary["lab_summary_json"] = labs_summary["lab_summary_json"].apply(labs_last_only)
else:
    labs_summary = labs_summary_full

# Microbiology (구조화 JSON)
if USE_MICROBIO:
    print("📥 Loading microbiologyevents ...")
    micro = safe_read_csv(
        f"{HOSP_DIR}/microbiologyevents.csv",
        usecols=["microevent_id","subject_id","hadm_id","micro_specimen_id","chartdate","charttime",
                 "spec_itemid","spec_type_desc","test_itemid","test_name","org_itemid","org_name",
                 "isolate_num","quantity","ab_itemid","ab_name","dilution_text","dilution_comparison","dilution_value","interpretation","comments"]
    )
    micro["charttime"] = td(micro["charttime"])
    micro = micro[micro["hadm_id"].isin(core["hadm_id"])].copy()

    def summarize_micro(g):
        res = []
        for (spec, org), gg in g.groupby(["spec_type_desc","org_name"], dropna=False):
            inter = gg["interpretation"].dropna().value_counts().to_dict()
            res.append({"specimen": spec, "organism": org, "n": int(len(gg)), "interpretation_cnt": inter})
        return jdump(res)
    micro_summary = (micro.groupby("hadm_id", group_keys=False)
                        .apply(lambda g: pd.Series({"microbio_summary_json": summarize_micro(g)}))
                        .reset_index())
else:
    micro_summary = pd.DataFrame(columns=["hadm_id","microbio_summary_json"])

# === 6) ICU 경량 ==============================================================
if USE_ICU_LIGHT:
    print("📥 Loading icustays (light) ...")
    icu = safe_read_csv(
        f"{ICU_DIR}/icustays.csv",
        usecols=["subject_id","hadm_id","stay_id","first_careunit","last_careunit","intime","outtime","los"]
    )
    for c in ["intime","outtime"]:
        icu[c] = td(icu[c])
    icu_agg = (icu.groupby("hadm_id", as_index=False)
                  .agg(icu_stay_count=("stay_id","count"),
                       icu_first_intime=("intime","min"),
                       icu_last_outtime=("outtime","max"),
                       icu_total_los_days=("los","sum")))
else:
    icu_agg = pd.DataFrame(columns=["hadm_id","icu_stay_count","icu_first_intime","icu_last_outtime","icu_total_los_days"])

# === 7) 조립 & A-입력 화이트리스트 ===========================================
print("🧩 Assembling (A-input) ...")
wide = (core
    .merge(presc_summary, on="hadm_id", how="left")
    .merge(emar_summary, on="hadm_id", how="left")
    .merge(labs_summary, on="hadm_id", how="left")
    .merge(micro_summary, on="hadm_id", how="left")
    .merge(icu_agg, on="hadm_id", how="left")
)

# 날짜 문자열화
date_cols = ["admittime","dischtime","deathtime","edregtime","edouttime","icu_first_intime","icu_last_outtime"]
for c in date_cols:
    if c in wide.columns:
        wide[c] = wide[c].dt.tz_convert(None).astype(str).replace("NaT","")

# ✅ A-입력에 남길 컬럼(구조화만)
INPUT_KEEP = [
    "subject_id","hadm_id",
    "gender","anchor_age","anchor_year_group",
    "admission_type","admission_location","discharge_location","insurance",
    "admittime","dischtime","hospital_expire_flag",
    "icu_stay_count","icu_total_los_days",
    "inpatient_med_summary_json",
    "lab_summary_json",
    "microbio_summary_json",
]
if USE_EMAR:
    INPUT_KEEP.append("emar_admin_summary_json")

input_A = wide[[c for c in INPUT_KEEP if c in wide.columns]].copy()

print(f"✅ A-input rows: {len(input_A):,}, cols: {len(input_A.columns)}")
in_base = os.path.join(OUTPUT_DIR, "mimiciv_record_sheet")

if SAVE_PARQUET:
    input_A.to_parquet(in_base + ".parquet", index=False)
    print(f"💾 Saved Parquet → {in_base}.parquet")
if SAVE_CSV:
    input_A.to_csv(in_base + ".csv", index=False)
    print(f"💾 Saved CSV     → {in_base}.csv")

print("🎉 Done (A-input)!")


📥 Loading patients/admissions ...
✅ core rows: 3
📥 Loading prescriptions ...


/tmp/ipython-input-510581600.py:109: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({"inpatient_med_summary_json": one_hadm(g)}))


📥 Loading d_labitems & labevents (filtered) ...


/tmp/ipython-input-510581600.py:199: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({"lab_summary_json": one_hadm(g)}))


📥 Loading microbiologyevents ...


/tmp/ipython-input-510581600.py:249: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({"microbio_summary_json": summarize_micro(g)}))


📥 Loading icustays (light) ...
🧩 Assembling (A-input) ...
✅ A-input rows: 3, cols: 17
💾 Saved Parquet → /content/drive/MyDrive/DILAB/MARS/mimic-iv_reconstructed/mimiciv_A_input_struct_only.parquet
🎉 Done (A-input)!


# 2. 출력(퇴원 요약) 데이터셋 파일 저장

In [ ]:
# === Discharge 전용 (output: discharge_record_md만) =========================
import os, re, json
import pandas as pd
import numpy as np

# 경로 ─────────────────────────────────────────────────────────────────────────
NOTE_DIR   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note"
OUTPUT_DIR = "/content/drive/MyDrive/DILAB/MARS/mimic-iv_reconstructed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 🔗 입력 파일과 매칭(있으면 해당 hadm만)
MASTER_PARQUET_PATH = os.path.join(OUTPUT_DIR, "mimiciv_record_sheet.parquet")
FILTER_TO_MASTER_HADM = True

OUT_PARQUET = os.path.join(OUTPUT_DIR, "mimiciv_note_record_sheet.parquet")
OUT_CSV     = os.path.join(OUTPUT_DIR, "mimiciv_note_record_sheet.csv")
SAVE_CSV    = False

def td(x): return pd.to_datetime(x, errors="coerce", utc=True)

# 섹션 파싱(백업용) 패턴 (정규식 오탈자 수정: {0,60})
SECTION_PATTERNS = [
    ("chief_complaint", r"(?:^|\n)\s*(?:chief complaint|cc)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("hpi",              r"(?:^|\n)\s*(?:history of present illness|hpi)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("pmh",              r"(?:^|\n)\s*(?:past medical history|pmh)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("fhx",              r"(?:^|\n)\s*(?:family history)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("shx",              r"(?:^|\n)\s*(?:social history)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("allergies",        r"(?:^|\n)\s*(?:allergies?)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("physical_exam",    r"(?:^|\n)\s*(?:physical (?:exam|examination))\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("assessment",       r"(?:^|\n)\s*(?:assessment|impression)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("hospital_course",  r"(?:^|\n)\s*(?:hospital course)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("discharge_diagnosis", r"(?:^|\n)\s*(?:discharge diagnosis(?:es)?)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("plan",             r"(?:^|\n)\s*(?:plan|discharge plan)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("discharge_instructions", r"(?:^|\n)\s*(?:discharge instructions?)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
    ("meds_on_discharge",r"(?:^|\n)\s*(?:medications? on discharge|discharge medications?)\s*[:\-]\s*(.*?)(?=\n[A-Z][^\n]{0,60}\s*:|\Z)"),
]
SECTION_COLS = [
    "chief_complaint","hpi","pmh","fhx","shx","allergies",
    "physical_exam","assessment","hospital_course","discharge_diagnosis",
    "plan","discharge_instructions","meds_on_discharge","assessment_plan"
]
LABELS = {
    "chief_complaint":"주호소","hpi":"현병력","pmh":"과거력","fhx":"가족력","shx":"사회력",
    "allergies":"알레르기","physical_exam":"신체검진","assessment":"평가/인상",
    "hospital_course":"입원경과","discharge_diagnosis":"퇴원진단",
    "plan":"치료계획","discharge_instructions":"퇴원지시","meds_on_discharge":"퇴원약",
    "assessment_plan":"평가/계획"
}

def fallback_parse_sections(text: str) -> dict:
    if not isinstance(text, str) or not text.strip():
        return {}
    t = text.lower()
    out = {}
    for key, pat in SECTION_PATTERNS:
        m = re.search(pat, t, flags=re.S|re.I)
        if m:
            out[key] = m.group(1).strip()
    return out

def compose_discharge_md(row: pd.Series) -> str:
    parts = [f"# 퇴원기록지 (subject_id={row['subject_id']}, hadm_id={row['hadm_id']})"]
    for k in [
        "chief_complaint","hpi","pmh","fhx","shx","allergies",
        "physical_exam","assessment","hospital_course","discharge_diagnosis",
        "plan","discharge_instructions","meds_on_discharge"
    ]:
        val = row.get(k)
        if pd.notna(val) and str(val).strip():
            parts.append(f"## {LABELS.get(k,k)}\n{str(val).strip()}")
    return "\n\n".join(parts)

# 1) discharge 원문 로드
dis = pd.read_csv(
    os.path.join(NOTE_DIR, "discharge.csv"),
    usecols=["note_id","subject_id","hadm_id","note_type","note_seq","charttime","storetime","text"],
    low_memory=False
)
dis["charttime"] = td(dis["charttime"]); dis["storetime"] = td(dis["storetime"])

# (옵션) A-입력과 hadm 동기화
if FILTER_TO_MASTER_HADM and os.path.exists(MASTER_PARQUET_PATH):
    master = pd.read_parquet(MASTER_PARQUET_PATH, columns=["hadm_id"])
    keep = set(master["hadm_id"].dropna().unique().tolist())
    dis = dis[dis["hadm_id"].isin(keep)].copy()

# hadm별 최신본
dis_latest = (dis.sort_values(["hadm_id","note_seq","charttime"])
                .drop_duplicates(subset=["hadm_id"], keep="last")
                .reset_index(drop=True))

# 2) detail로 섹션 채우기(가능하면)
detail_path = os.path.join(NOTE_DIR, "discharge_detail.csv")
if os.path.exists(detail_path):
    det = pd.read_csv(
        detail_path,
        usecols=["note_id","subject_id","field_name","field_value","field_ordinal"],
        low_memory=False
    ).sort_values(["note_id","field_name","field_ordinal"])

    agg = (det.groupby(["note_id","field_name"], as_index=False)["field_value"]
              .apply(lambda s: "\n".join([str(x) for x in s if pd.notna(x)])))

    FIELD_MAP = {
        "Chief Complaint":"chief_complaint",
        "History of Present Illness":"hpi",
        "Past Medical History":"pmh",
        "Family History":"fhx",
        "Social History":"shx",
        "Allergies":"allergies",
        "Physical Exam":"physical_exam",
        "Assessment":"assessment",
        "Impression":"assessment",
        "Hospital Course":"hospital_course",
        "Discharge Diagnosis":"discharge_diagnosis",
        "Plan":"plan",
        "Assessment and Plan":"assessment_plan",
        "Discharge Instructions":"discharge_instructions",
        "Medications on Discharge":"meds_on_discharge",
        "Discharge Medications":"meds_on_discharge",
    }
    agg["std_key"] = agg["field_name"].map(FIELD_MAP).fillna(agg["field_name"])
    piv = agg.pivot(index="note_id", columns="std_key", values="field_value").reset_index()
    dis_latest = dis_latest.merge(piv, on="note_id", how="left")

# 섹션 dtype 정리
for col in SECTION_COLS:
    if col not in dis_latest.columns:
        dis_latest[col] = pd.Series(index=dis_latest.index, dtype="string")
    else:
        dis_latest[col] = dis_latest[col].astype("string")

# 3) detail 비어있으면 원문에서 백업 파싱
mask_need = dis_latest[SECTION_COLS].isna().all(axis=1)
if mask_need.any():
    parsed_df = (pd.json_normalize(dis_latest.loc[mask_need, "text"].apply(fallback_parse_sections))
                   .reindex(columns=SECTION_COLS).astype("string"))
    parsed_df.index = dis_latest.index[mask_need]
    sub = dis_latest.loc[mask_need, SECTION_COLS]
    dis_latest.loc[mask_need, SECTION_COLS] = sub.where(sub.notna(), parsed_df)

# 4) MD 한 장 생성
dis_latest["discharge_record_md"] = dis_latest.apply(compose_discharge_md, axis=1)

# 5) 🔥 A-출력 최소 컬럼만 저장
out_A = dis_latest[["subject_id","hadm_id","discharge_record_md"]].copy()

out_A.to_parquet(OUT_PARQUET, index=False)
if SAVE_CSV:
    out_A.to_csv(OUT_CSV, index=False)

print(f"✅ rows: {len(out_A):,} | cols: {len(out_A.columns)}")
print(f"💾 Saved: {OUT_PARQUET}")
if SAVE_CSV:
    print(f"💾 Saved: {OUT_CSV}")


✅ rows: 3 | cols: 3
💾 Saved: /content/drive/MyDrive/DILAB/MARS/mimic-iv_reconstructed/discharge_records_A_output.parquet
